# **鲁棒优化笔记：静态鲁棒与两阶段鲁棒优化**

## 1. 静态鲁棒优化

### 1.1 原始问题
$$\max_{x} \min_{u \in U} c^T x$$
$$\text{s.t. } Ax \geq b - Bu, \quad x \geq 0, \ u \in U$$

可以通过KKT条件转化为单层优化问题，或者用强对偶定理处理。

### **1.2 KKT条件**

#### 步骤1：固定外层变量$u$，对内层问题用KKT条件
内层问题：
$$\min_{x} c^T x$$
$$\text{s.t. } Ax \geq b - Bu, \quad x \geq 0$$

#### 步骤2：引入拉格朗日乘子，构造拉格朗日函数
$$L(x,y,w) = c^T x - y^T (Ax - (b - Bu)) - w^T x$$
其中：
- $y \geq 0$：约束$Ax \geq b - Bu$的乘子
- $w \geq 0$：约束$x \geq 0$的乘子

#### 步骤3：列出KKT条件及原问题约束
1. **原始可行**：
   $$Ax \geq b - Bu, \quad x \geq 0$$

2. **拉格朗日对$x$梯度为0**：
   $$\nabla_x L = -c - A^T y - w = 0 \Rightarrow A^T y + w = -c$$

   也可以从对偶问题的角度理解该条件，对偶问题：
   $$\max (b-Bu)y$$
   $$\text{s.t. }A^T y \leq c, \quad y \geq 0$$
   对于对偶问题的不等式$A^T y \leq c$，引入松弛变量$w \geq 0$化为等式$A^T y + w \leq c$，与拉格朗日函数对$x$梯度为0的条件等价。

3. **拉格朗日乘子非负**
   $$y \geq 0, \ w \geq 0$$

4. **互补松弛条件**：
   $$y^T (Ax - (b - Bu)) = 0$$
   $$w^T x = 0$$

#### 步骤4：将原问题内层替换为KKT条件，得到单层优化问题
$$\max_{x,y,w,u} c^T x$$
$$\begin{align*}
\text{s.t. } & Ax \geq b - Bu \\
& x \geq 0 \\
& A^T y + w = -c \\
& y \geq 0, \ w \geq 0 \\
& y^T (Ax - (b - Bu)) = 0 \\
& w^T x = 0 \\
& u \in U
\end{align*}$$

#### 步骤5：互补松弛条件线性化（大M法）
互补松弛条件包含两个变量的乘积（$y^T x$与$w^T x$），为非线性项，引入二元变量$p_i, q_j \in \{0,1\}$与很大的常数$M$线性化：
$$\begin{cases}
0 \leq y_i \leq M p_i \\
0 \leq (Ax - (b - Bu))_i \leq M (1-p_i) \\
0 \leq w_j \leq M q_j \\
0 \leq x_j \leq M (1-q_j)
\end{cases}$$

#### MILP模型
$$\max_{x,y,w,u,p,q} c^T x$$
$$\begin{align*}
\text{s.t. } & Ax \geq b - Bu \\
& x \geq 0 \\
& A^T y + w = -c \\
& y \geq 0, \ w \geq 0 \\
& u \in U \\
& 0 \leq y_i \leq M p_i \quad \forall i \\
& 0 \leq (Ax - (b - Bu))_i \leq M (1-p_i) \quad \forall i \\
& 0 \leq w_j \leq M q_j \quad \forall j \\
& 0 \leq x_j \leq M (1-q_j) \quad \forall j \\
& p_i, q_j \in \{0,1\} \quad \forall i,j
\end{align*}$$


### 1.3 强对偶定理
#### 步骤1：替换内层问题为对偶问题
原始问题：
$$\min_{x} c^T x \quad \text{s.t.} \quad Ax \geq b - Bu, \quad x \geq 0$$
对偶问题：
$$\max_{y} (b - Bu)^T y \quad \text{s.t.} \quad A^T y \leq c, \quad y \geq 0$$

#### 步骤2：双层优化合并

$$\max_{u \in U} \max_{y} (b - Bu)^T y \quad \text{s.t.} \quad A^T y \leq c, \quad y \geq 0$$

等价于单层问题：

$$\max_{u \in U, y} (b - Bu)^T y \quad \text{s.t.} \quad A^T y \leq c, \quad y \geq 0$$

#### 步骤3：双线性项处理
合并后的目标函数：
$$(b - Bu)^T y = b^T y - u^T (B^T y)$$

其中 $u^T (B^T y)$ 是双线性项。令 $v = B^T y$，则双线性项为 $u^T v$

采用**变量分解**：
$$u = \bar{u} + \delta u(z^+-z^-)$$
即将不确定量$u$表示为名义值与偏差值，$\bar{u}$和$\delta u$均为常数，则：
$$u^T v = \bar{u}^T v + \delta u^T v^+ - \delta u^T v^-$$
最终目标函数为：
$$\max_{y, v, z} b^T y - u^T v - \delta \bar{u}^T v^+ + \delta \bar{u}^T v^-$$

**约束设计**：
- 大M法线性化：
  
$v_j^+ = z_j^+ v_j$相当于：
$$\begin{cases}
0 \leq v_j^+ - v_j \leq M(1 - z_j^+) \\
-M z_j^+ \leq v_j^+ \leq 0 
\end{cases} \quad \forall j$$
同理处理$v_j^- = z_j^- v_j$。

- 逻辑约束：
$$z_j^+ + z_j^- \leq 1 \quad (\text{二元变量互斥})$$
$$ \sum_j (z_j^+ + z_j^-)\leq \Gamma \quad (\text{预算上限})$$

#### 完整线性化模型

$$\max_{y, v, z} b^T y - u^T v - \delta u^T v^+ + \delta u^T v^-$$

$$\begin{align*}
\text{s.t. } & A^T y \leq c \\
&v = B^T y \\
& 0 \leq v_j^+ - v_j \leq M(1 - z_j^+)  \quad \forall j \\
& -M z_j^+ \leq v_j^+ \leq 0 \quad \forall j \\
& 0 \leq v_j^- - v_j \leq M(1 - z_j^-)  \quad \forall j \\
& -M z_j^- \leq v_j^- \leq 0 \quad \forall j \\
& z_j^* + z_j^* \leq 1 \quad \forall j \\
& \sum_j (z_j^+ + z_j^-)\leq \Gamma \\
& v^+, v^- \in \mathbb{R}^n \\
& y \geq 0, v \geq 0, z_j^* \in \{0,1\}
\end{align*}$$

#### 关键技巧总结
1. **对偶转化**：将min-max问题转化为单层max问题
2. **双线性分解**：$u^T v = \bar{u}^T v + \delta u^T v^+ - \delta u^T v^-$
3. **线性化**：使用大M法和二元变量$z_j^*$处理非线性项

## **2. 两阶段鲁棒优化**

### 标准模型
通用的数学模型如下所示：
$$\min_{y} c^{T}y + \max_{u \in \mathcal{U}} \min_{x \in F(y, u)} b^{T}x$$
约束条件为：
$$Ay \ge d \quad \text{(1)}$$
$$Gx \ge h - Ey - Mu, \quad \forall u \in \mathcal{U} \quad \text{(2)}$$
$$y \in S_{y} \subseteq \mathbb{R}_{+}^{n}, \quad x \in S_{x} \subseteq \mathbb{R}_{+}^{m}$$

其中：
* $y$ 和 $x$ 分别是第一和第二阶段的决策变量。
* $u$ 是不确定参数，其取值范围由不确定集 $\mathcal{U}$ 描述。
* 目标函数的第一部分 $c^Ty$ 是第一阶段成本，第二部分 $\max_{u \in \mathcal{U}} \min_{x \in F(y, u)} b^{T}x$ 是在最坏情况下的第二阶段成本。
由于不确定集 $\mathcal{U}$ 通常包含大量甚至无穷多个场景，直接枚举求解是不可行的，因此需要设计专门的算法。

参考文献介绍了**Benders-dual cutting plane method**和**Column and constraint method**两种方法，将原问题分解为主问题和子问题，对子问题（min问题）的内层取对偶（max问题）的操作是一样的。

Benders分解之后将对偶子问题的内外层（均为max问题）合并，将子问题转为单层优化问题，是双线性问题，可用启发式算法/Gurobi/KKT条件求解。求解子问题的目标函数为主问题的辅助变量$\eta$提供下界，即构造最优割。$\min_{y} c^T y + \max_{u \in U} \min_{x \in F(y,u)} b^T x$为原问题提供上界UB，$\min_{y} c^T y + \eta$为原问题提供下界LB，上下界不断改进，知道LB=UB。

CCG算法则对子问题内层用**KKT条件**替代，将子问题转化为单层优化问题。求解子问题，将可能是最坏场景的不确定量对应的**决策变量和约束**加回到主问题中。

| 特性                | CCG算法                     | Benders-Dual算法          |
|---------------------|----------------------------|--------------------------|
| **变量管理**        | 每次迭代增加新变量$x^k$     | 决策变量不变             |
| **第一阶段要求**    | 支持整数变量               | 仅支持线性规划           |
| **收敛速度**        | $O(p)$                     | $O(p \times q)$          |
| **子问题类型**      | 双线性规划                 | 线性规划                 |
| **计算复杂度**      | 较低                       | 较高                     |
| **实现难度**        | 中等                       | 较低                     |
| **适用场景**        | 复杂不确定性集合           | 简单不确定性集合         |



### **列与约束生成（C&CG）算法详解**

> **参考**  
> Zeng B., Zhao L. (2013). *Solving two-stage robust optimization problems using a column-and-constraint generation method*. **Operations Research Letters**.  
> 以及公众号文章《C&CG算法求解 Two-stage RO 的完整推导与代码》。

#### 1. 主问题 (Master Problem)

主问题用于求解第一阶段变量 $y$，同时利用一个辅助变量 $\eta$ 来近似最坏情况下的第二阶段成本。在C&CG算法的第 $k$ 次迭代中，主问题 $MP_2$ 的形式如下：
$$MP_2: \min_{y, \eta, \{x^l\}} c^{T}y + \eta$$
约束条件为：
$$Ay \ge d$$
$$\eta \ge b^{T}x^l, \quad l=1,...,r$$
$$Ey + Gx^l \ge h - Mu_l^*, \quad l=1,...,r$$
$$y \in S_y, \quad \eta \in \mathbb{R}, \quad x^l \in S_x, \quad l=1,...,r$$
* 每次迭代，算法会从子问题中得到一个最坏场景 $u_k^*$。
* 随后，一组新的第二阶段变量 $x^k$ (即“列”)和与之相关的约束被添加到主问题中。这使得主问题对第二阶段成本的估计越来越精确，其目标函数值是原问题的一个下界（Lower Bound）。

#### 2. 子问题 (Subproblem)

子问题的目标是，对于主问题给出的一个固定的第一阶段解 $\bar{y}$，在不确定集 $\mathcal{U}$ 中找到一个能导致第二阶段成本最大的“最坏”场景 $u^*$。子问题 $SP_2$ 的形式是一个双层优化问题：
$$SP_2: Q(\bar{y}) = \max_{u \in \mathcal{U}} \min_{x} \{ b^{T}x : Gx \ge h - E\bar{y} - Mu, x \in S_x \}$$
这个双层结构是求解的难点。C&CG算法通过对内层的最小化问题进行对偶变换或使用KKT条件，将其转化为单层问题。

#### 3. 子问题的KKT条件转化

为了求解子问题 $SP_2$，我们可以利用其内层问题是一个线性规划的特性。假设强对偶性成立（例如，满足Slater's条件），我们可以用KKT（Karush-Kuhn-Tucker）条件来等价替换内层的最小化问题。

对于给定的 $y$ 和 $u$，内层问题为：
$$\min_{x} b^{T}x$$
约束条件：
$$h - Ey - Mu - Gx \le 0$$
$$-x \le 0$$
通过引入对偶变量 $\pi$ 和 $\lambda$，其KKT条件包括原始可行性、对偶可行性以及互补松弛条件。将这些KKT条件代入外层的最大化问题，子问题 $SP_2$ 就被转化为一个等价的单层优化问题：
$$\max \quad b^{T}x$$
约束条件：
$$Gx \ge h - Ey - Mu \quad \text{(原始可行性)}$$
$$G^{T}\pi \le b \quad \text{(对偶可行性)}$$
$$(Gx - h + Ey + Mu)_i \cdot \pi_i = 0, \quad \forall i \quad \text{(互补松弛)}$$
$$(b - G^{T}\pi)_j \cdot x_j = 0, \quad \forall j \quad \text{(互补松弛)}$$
$$u \in \mathcal{U}, \quad x \in S_x, \quad \pi \ge 0$$

#### 4. 等价线性化

转化后的子问题中包含了非线性的互补松弛约束（例如 $(b - G^{T}\pi)_j \cdot x_j = 0$）。这些约束可以通过引入辅助二元变量和“大M”（Big-M）方法进行线性化 。

例如，对于约束 $(b - G^{T}\pi)_j \cdot x_j = 0$，我们可以引入一个二元变量 $v_j \in \{0, 1\}$ 和一个足够大的常数 $M$，将其替换为以下两个线性约束：
$$x_j \le M \cdot v_j$$
$$(b - G^{T}\pi)_j \le M \cdot (1 - v_j)$$
通过这种方式，整个子问题就可以被转化为一个混合整数线性规划（MILP），从而可以使用Gurobi等商业求解器进行有效求解。

#### 5. 迭代流程

1.  **初始化**:
    * 设置上下界 $LB = -\infty$, $UB = +\infty$。
    * 构建一个初始的主问题，可能只包含第一阶段的变量和约束，或者包含一个初始的、预估的场景。

2.  **迭代循环**: 在第 $k$ 次迭代中：
    * **求解主问题**: 求解当前的主问题（Master Problem），得到第一阶段决策 $y_k^*$ 和一个当前最优的目标值。这个目标值是原问题真实最优解的一个**下界 (Lower Bound)**。
        $$LB = c^T y_k^* + \eta_k^*$$
    * **求解子问题**: 将主问题得到的 $y_k^*$ 固定，代入子问题（Subproblem）中进行求解。子问题的目标是找到在此 $y_k^*$ 下，能使第二阶段成本最大化的“最坏”不确定性场景 $u_k^*$ 以及对应的第二阶段决策 $x_k^*$ 和目标值 $Q(y_k^*)$。
    * **更新上界**: 利用子问题的解来更新全局的**上界 (Upper Bound)**。上界是目前为止我们找到了一个可行解所对应的真实目标函数值。
        $$UB = \min(UB, \ c^T y_k^* + Q(y_k^*))$$
    * **检查收敛**: 判断上下界的差距是否满足收敛条件。
        $$\text{If } (UB - LB) / UB \le \epsilon \text{, then terminate.}$$
        如果满足，则当前解 $y_k^*$ 就是最优解，算法结束。

    * **生成列与约束**: 如果未收敛，则需要利用刚刚从子问题中得到的最坏场景 $u_k^*$ 来加强主问题：
        * **生成新列 (Column Generation)**: 在主问题中，创建一组全新的第二阶段决策变量，记为 $x^{k+1}$。这些变量代表了在第 $k+1$ 个被识别出的最坏场景下的应对策略。
        * **生成新约束 (Constraint Generation)**: 将与新变量 $x^{k+1}$ 和最坏场景 $u_k^*$ 相关的约束添加到主问题中。主要包括两种：
            1.  **优化性割平面 (Optimality Cut)**: 这个约束将辅助变量 $\eta$ 与新场景的第二阶段成本关联起来，确保 $\eta$ 能够正确地反映所有已知最坏场景中的最大成本。
                $$\eta \ge b^T x^{k+1}$$
            2.  **可行性约束 (Feasibility Constraints)**: 这组约束确保了对于场景 $u_k^*$，第一阶段决策 $y$ 和新的第二阶段决策 $x^{k+1}$ 必须是可行的。
                $$Gy^{k+1} \ge h - Ey - Mu_k^*$$


#### 6. 案例分析: 鲁棒选址-运输问题

文档中以一个经典的选址-运输问题作为案例，来演示C&CG算法的应用。

* **问题背景**：
    * 第一阶段：决定仓库的建设与否 ($y_i$) 及其容量 ($z_i$)。
    * 第二阶段：在需求明确后，安排从已建仓库到客户的运输方案 ($x_{ij}$) 。
    * 不确定性：客户的需求 $d_j$ 是不确定的，在一个预定义的多面体不确定集 $D$ 内波动。

* **原问题模型**
该问题的两阶段鲁棒优化模型如下：
$$\min_{y,z} \sum_{i} f_i y_i + \sum_{i} c_i z_i + \max_{d \in D} \min_{x} \sum_{i,j} t_{ij} x_{ij}$$
第一阶段约束包括：
$$z_i \le K_i y_i, \quad \forall i$$
第二阶段约束包括：
$$\sum_{j} x_{ij} \le z_i, \quad \forall i$$
$$\sum_{i} x_{ij} \ge d_j, \quad \forall j$$
不确定集 $D$ 的定义为：
$$D = \{d : d_j = \bar{d_j} + \xi_j g_j, \quad \sum g_j \le \Gamma, \dots \}$$

* **主问题与子问题**
- **主问题**：求解第一阶段变量 $y, z$ 和第二阶段成本的近似值 $\eta$。每次迭代，会根据子问题找到的最坏需求场景 $d^k$ 和对应的运输变量 $x^k$，添加新的列和约束。
- **子问题**：对于给定的 $y, z$，找到最坏的需求 $d \in D$，使得第二阶段的运输成本最小化的值达到最大。
    $$
    \max_{d \in D} \min_{x \ge 0} \{\sum_{i,j} t_{ij} x_{ij} \mid \sum_{j} x_{ij} \le z_i, \sum_{i} x_{ij} \ge d_j \}
    $$

#### 7. 代码

In [1]:
from gurobipy import *
import numpy as np

""" The input parameter """
facility_num = 3
customer_num = 3
fixed_cost = [400, 414, 326]
unit_capacity_cost = [18, 25, 20]
trans_cost = [[22, 33, 24],
              [33, 23, 30],
              [20, 25, 27]]
max_capacity = 800

demand_nominal = [206, 274, 220]
demand_var = [40, 40, 40]

""" build initial master problem """
""" Create variables """
master = Model('master problem')
x_master = {}
z = {}
y = {}
eta = master.addVar(lb=0, vtype=GRB.CONTINUOUS, name='eta')

for i in range(facility_num):
    y[i] = master.addVar(vtype=GRB.BINARY, name='y_%s' % (i))
    z[i] = master.addVar(lb=0, vtype=GRB.CONTINUOUS, name='z_%s' % (i))

""" Set objective """
obj = LinExpr()
for i in range(facility_num):
    obj.addTerms(fixed_cost[i], y[i])
    obj.addTerms(unit_capacity_cost[i], z[i])
obj.addTerms(1, eta)

master.setObjective(obj, GRB.MINIMIZE)

""" Add Constraints  """
# cons 1
for i in range(facility_num):
    master.addConstr(z[i] <= max_capacity * y[i])

""" Add initial value Constraints  """
# create new variables x
iter_cnt = 0
for i in range(facility_num):
    for j in range(customer_num):
        x_master[iter_cnt, i, j] = master.addVar(lb=0
                                                 , ub=GRB.INFINITY
                                                 , vtype=GRB.CONTINUOUS
                                                 , name='x_%s_%s_%s' % (iter_cnt, i, j))
# create new constraints
expr = LinExpr()
for i in range(facility_num):
    for j in range(customer_num):
        expr.addTerms(trans_cost[i][j], x_master[iter_cnt, i, j])
master.addConstr(eta >= expr)

expr = LinExpr()
for i in range(facility_num):
    expr.addTerms(1, z[i])
master.addConstr(expr >= 772)  # 206 + 274 + 220 + 40 * 1.8

""" solve the model and output """
master.optimize()
print('Obj = {}'.format(master.ObjVal))
print('-----  location ----- ')
for key in z.keys():
    print('facility : {}, location: {}, capacity: {}'.format(key, y[key].x, z[key].x))


def print_sub_sol(model, d, g, x):
    d_sol = {}
    if model.status != GRB.OPTIMAL:
        print('The problem is infeasible or unbounded!')
        print('Status: {}'.format(model.status))
        d_sol[0] = 0
        d_sol[1] = 0
        d_sol[2] = 0
    else:
        print('Obj(sub) : {:.2f}'.format(model.ObjVal), end='\t | ')
        for key in d.keys():
            d_sol[key] = d[key].x
    return d_sol


""" Column-and-constraint generation """

LB = -np.inf
UB = np.inf
iter_cnt = 0
max_iter = 30
eps = 0.001
Gap = np.inf

z_sol = {}
for key in z.keys():
    z_sol[key] = z[key].x

""" solve the master problem and update bound """
master.optimize()

""" 
 Update the Lower bound 
"""
LB = master.ObjVal
print('LB: {:.2f}'.format(LB))

''' create the subproblem '''
subProblem = Model('sub problem')
x = {}  # transportation decision variables in subproblem
d = {}  # true demand
g = {}  # uncertainty part: var part
pi = {}  # dual variable
theta = {}  # dual variable
v = {}  # aux var for capacity constraint
w = {}  # aux var for demand constraint
h = {}  # aux var for transportation constraint
big_M = 10000  # Big M value for linearization

# Create variables
for i in range(facility_num):
    pi[i] = subProblem.addVar(lb=-GRB.INFINITY, ub=0, vtype=GRB.CONTINUOUS, name='pi_%s' % i)
    v[i] = subProblem.addVar(vtype=GRB.BINARY, name='v_%s' % i)
    
for j in range(customer_num):
    w[j] = subProblem.addVar(vtype=GRB.BINARY, name='w_%s' % j)
    g[j] = subProblem.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS, name='g_%s' % j)
    theta[j] = subProblem.addVar(lb=-GRB.INFINITY, ub=0, vtype=GRB.CONTINUOUS, name='theta_%s' % j)
    d[j] = subProblem.addVar(lb=0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name='d_%s' % j)
    
for i in range(facility_num):
    for j in range(customer_num):
        h[i, j] = subProblem.addVar(vtype=GRB.BINARY, name='h_%s_%s' % (i, j))
        x[i, j] = subProblem.addVar(lb=0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name='x_%s_%s' % (i, j))

""" set objective """
sub_obj = LinExpr()
for i in range(facility_num):
    for j in range(customer_num):
        sub_obj.addTerms(trans_cost[i][j], x[i, j])
subProblem.setObjective(sub_obj, GRB.MAXIMIZE)

""" add constraints to subproblem """
# cons 1: Capacity constraints
for i in range(facility_num):
    expr = LinExpr()
    for j in range(customer_num):
        expr.addTerms(1, x[i, j])
    subProblem.addConstr(expr <= z_sol[i], name='sub_capacity_%s' % i)

# cons 2: Demand satisfaction
for j in range(customer_num):
    expr = LinExpr()
    for i in range(facility_num):
        expr.addTerms(1, x[i, j])
    subProblem.addConstr(expr >= d[j], name='sub_demand_%s' % j)

# cons 3: Dual constraint
for i in range(facility_num):
    for j in range(customer_num):
        subProblem.addConstr(pi[i] - theta[j] <= trans_cost[i][j], 
                            name='dual_constraint_%s_%s' % (i, j))

""" demand constraints """
for j in range(customer_num):
    subProblem.addConstr(d[j] == demand_nominal[j] + g[j] * demand_var[j], 
                        name='demand_expr_%s' % j)

subProblem.addConstr(g[0] + g[1] + g[2] <= 1.8, name='g_sum1')
subProblem.addConstr(g[0] + g[1] <= 1.2, name='g_sum2')

""" logic constraints for KKT conditions """
# Logic 1: Capacity constraints complementary slackness
for i in range(facility_num):
    # π_i >= -M * v_i
    subProblem.addConstr(pi[i] >= -big_M * v[i], name='logic1a_%s' % i)
    # z_i - sum_j x_ij <= M * (1 - v_i)
    subProblem.addConstr(z_sol[i] - sum(x[i, j] for j in range(customer_num)) <= big_M * (1 - v[i]), 
                        name='logic1b_%s' % i)

# Logic 2: Demand constraints complementary slackness
for j in range(customer_num):
    # θ_j >= -M * w_j
    subProblem.addConstr(theta[j] >= -big_M * w[j], name='logic2a_%s' % j)
    # sum_i x_ij - d_j <= M * (1 - w_j)
    subProblem.addConstr(sum(x[i, j] for i in range(facility_num)) - d[j] <= big_M * (1 - w[j]), 
                        name='logic2b_%s' % j)

# Logic 3: Transportation constraints complementary slackness
for i in range(facility_num):
    for j in range(customer_num):
        # x_ij <= M * h_{ij}
        subProblem.addConstr(x[i, j] <= big_M * h[i, j], name='logic3a_%s_%s' % (i, j))
        # c_ij - π_i + θ_j <= M * (1 - h_{ij})
        subProblem.addConstr(trans_cost[i][j] - pi[i] + theta[j] <= big_M * (1 - h[i, j]), 
                            name='logic3b_%s_%s' % (i, j))

subProblem.update()
subProblem.write('SP.lp')
subProblem.optimize()
d_sol = {}

print('\n\n\n *******            C&CG starts          *******  ')
print('\n **                Initial Solution             ** ')

d_sol = print_sub_sol(subProblem, d, g, x)

""" 
 Update the initial Upper bound 
"""
if subProblem.status == GRB.OPTIMAL:
    UB = min(UB, subProblem.ObjVal + master.ObjVal - eta.x)
print('UB (iter {}): {:.2f}'.format(iter_cnt, UB))

# close the outputflag
master.setParam('Outputflag', 0)
subProblem.setParam('Outputflag', 0)

"""
 Main loop of CCG algorithm 
"""
while (UB - LB > eps and iter_cnt <= max_iter):
    iter_cnt += 1
    print('\n iter : {} '.format(iter_cnt), end='\t | ')

    # Create new variables for this iteration
    for i in range(facility_num):
        for j in range(customer_num):
            x_master[iter_cnt, i, j] = master.addVar(
                lb=0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, 
                name='x_%s_%s_%s' % (iter_cnt, i, j))

    # Get the worst-case demand from subproblem
    d_worst = [d[j].x for j in range(customer_num)]
    
    # If subproblem is feasible and bounded
    if subProblem.status == GRB.OPTIMAL:
        # Create new optimality cut: eta >= sum c_ij x_ij^l
        expr = LinExpr()
        for i in range(facility_num):
            for j in range(customer_num):
                expr.addTerms(trans_cost[i][j], x_master[iter_cnt, i, j])
        master.addConstr(eta >= expr, name='opt_cut_%s' % iter_cnt)

        # Add constraints for this scenario
        # Capacity constraints for this scenario
        for i in range(facility_num):
            expr = LinExpr()
            for j in range(customer_num):
                expr.addTerms(1, x_master[iter_cnt, i, j])
            master.addConstr(expr <= z[i], name='cap_scen_%s_%s' % (iter_cnt, i))
        
        # Demand constraints for this scenario
        for j in range(customer_num):
            expr = LinExpr()
            for i in range(facility_num):
                expr.addTerms(1, x_master[iter_cnt, i, j])
            master.addConstr(expr >= d_worst[j], name='dem_scen_%s_%s' % (iter_cnt, j))

        # Solve the master problem
        master.optimize()
        print('Obj(master): {:.2f}'.format(master.ObjVal), end='\t | ')
        
        """ Update the LB """
        LB = master.ObjVal
        print('LB: {:.2f}'.format(LB), end='\t | ')
        
        """ Update the subproblem """
        # Update z_sol from master solution
        for key in z.keys():
            z_sol[key] = z[key].x
        
        # Update capacity constraints in subproblem
        for i in range(facility_num):
            # Remove old constraint
            constr_name = 'sub_capacity_%s' % i
            constr = subProblem.getConstrByName(constr_name)
            if constr:
                subProblem.remove(constr)
            
            # Add new constraint with updated z_sol
            expr = LinExpr()
            for j in range(customer_num):
                expr.addTerms(1, x[i, j])
            subProblem.addConstr(expr <= z_sol[i], name=constr_name)
        
        # Re-optimize subproblem
        subProblem.optimize()
        d_sol = print_sub_sol(subProblem, d, g, x)
        
        """ Update the Upper bound """
        if subProblem.status == GRB.OPTIMAL:
            total_cost = sum(fixed_cost[i] * y[i].x for i in range(facility_num))
            total_cost += sum(unit_capacity_cost[i] * z[i].x for i in range(facility_num))
            total_cost += subProblem.ObjVal
            
            UB = min(UB, total_cost)
        
        print('UB: {:.2f}'.format(UB), end='\t | ')
        Gap = 100 * (UB - LB) / UB if UB != 0 else 100
        print('Gap: {:.2f}%'.format(Gap), end='\t')
        
    # If the subproblem is infeasible (shouldn't happen with relative complete recourse)
    else:
        print('Subproblem infeasible! Adding feasibility cut.')
        # Create feasibility cut (not needed in this case study)
        # For completeness, we add the scenario constraints
        for j in range(customer_num):
            expr = LinExpr()
            for i in range(facility_num):
                expr.addTerms(1, x_master[iter_cnt, i, j])
            master.addConstr(expr >= d_worst[j], name='feas_cut_%s_%s' % (iter_cnt, j))
        
        master.optimize()
        print('Obj(master): {:.2f}'.format(master.ObjVal))
        LB = master.ObjVal
        print('LB: {:.2f}'.format(LB))
        
        # Update subproblem with new z
        for key in z.keys():
            z_sol[key] = z[key].x
        
        for i in range(facility_num):
            constr_name = 'sub_capacity_%s' % i
            constr = subProblem.getConstrByName(constr_name)
            if constr:
                subProblem.remove(constr)
            
            expr = LinExpr()
            for j in range(customer_num):
                expr.addTerms(1, x[i, j])
            subProblem.addConstr(expr <= z_sol[i], name=constr_name)
        
        subProblem.optimize()
        d_sol = print_sub_sol(subProblem, d, g, x)
        
        if subProblem.status == GRB.OPTIMAL:
            total_cost = sum(fixed_cost[i] * y[i].x for i in range(facility_num))
            total_cost += sum(unit_capacity_cost[i] * z[i].x for i in range(facility_num))
            total_cost += subProblem.ObjVal
            UB = min(UB, total_cost)
        
        print('UB: {:.2f}'.format(UB))
        Gap = 100 * (UB - LB) / UB if UB != 0 else 100
        print('Gap: {:.2f}%'.format(Gap))

# Final output
master.write('finalMP.lp')
print('\n\nOptimal solution found!')
print('Optimal Obj: {:.2f}'.format(master.ObjVal))
print(' ** Final Gap: {:.2f}% ** '.format(Gap))
print('\n ** Facility Solution ** ')
for i in range(facility_num):
    print('Facility {}: Open={}, Capacity={:.2f}'.format(i, int(y[i].x), z[i].x))

print('\n ** Worst-case Demand ** ')
for j in range(customer_num):
    print('Customer {}: Nominal={}, Worst-case={:.2f}, Perturbation={:.2f}'.format(
        j, demand_nominal[j], d[j].x, g[j].x))

print('\n ** Transportation Solution ** ')
total_trans_cost = 0
for i in range(facility_num):
    for j in range(customer_num):
        if x[i, j].x > 1e-4:  # Only print non-zero flows
            cost = trans_cost[i][j] * x[i, j].x
            total_trans_cost += cost
            print('From facility {} to customer {}: {:.2f} units, Cost={:.2f}'.format(
                i, j, x[i, j].x, cost))

print('\nTotal transportation cost: {:.2f}'.format(total_trans_cost))
print('Total fixed cost: {:.2f}'.format(
    sum(fixed_cost[i] * y[i].x for i in range(facility_num))))
print('Total capacity cost: {:.2f}'.format(
    sum(unit_capacity_cost[i] * z[i].x for i in range(facility_num))))
print('Total cost: {:.2f}'.format(total_trans_cost+sum(fixed_cost[i] * y[i].x for i in range(facility_num))+sum(unit_capacity_cost[i] * z[i].x for i in range(facility_num))))

Set parameter LicenseID to value 2633037
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 5 rows, 16 columns and 19 nonzeros
Model fingerprint: 0x917392b8
Variable types: 13 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [1e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+02, 8e+02]
Presolve removed 1 rows and 10 columns
Presolve time: 0.01s
Presolved: 4 rows, 6 columns, 9 nonzeros
Variable types: 0 continuous, 6 integer (3 binary)
Found heuristic solution: objective 15036.000000

Root relaxation: objective 1.429600e+04, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Ga